generate varfile

In [24]:
# swlist = range(40,121,20)

numsw = 120
factor = 64

numserver = (numsw//5)*(numsw//5)*12
numport = int(numsw*0.8)
numleaf = int(numsw*0.8)
numspine = numsw-numleaf
varfile = f"/home/annzhou/DRing/src/emp/datacentre/qvarfiles/var_leafspine_{numserver}_{numsw}_{numport}_0_0_ecmp_{factor}"
with open(varfile,'w') as f:
    for srcsw in range(numleaf):
        for dstsw in range(numleaf):
            if srcsw == dstsw: continue
            weight = 1.0/numspine
            for pid in range(numspine):
                f.write(f"{srcsw}_{dstsw}_{pid},{weight}\n")

generate qvarfile

In [25]:
import random

def quantize_to_multiple_of_1_64(x,factor):
    return round(x * factor) / factor

def quantize_and_adjust_with_indices(indexed_values,factor,rseed,isprint=False):
    if isprint: print(indexed_values)
    # Quantize each value
    quantized_values = [(i, quantize_to_multiple_of_1_64(v,factor)) for i, v in indexed_values]
    if isprint: print(quantized_values)
    
    # Calculate the current sum
    current_sum = sum(v for _, v in quantized_values)
    
    # Calculate the difference between the sum and 1
    difference = current_sum - 1
    if isprint: print(f"currsum={current_sum}, difference={difference}")

    random.seed(rseed)
    random_values = [(v, random.random()) for v in quantized_values]
    if isprint: print(random_values)
    
    if difference != 0:
        if difference > 0:
            # Sort indices by quantized values to adjust the largest values first
            sorted_random = sorted(random_values, key=lambda x: (x[0][1],x[1]), reverse=True)
            # sorted_quantized = sorted(quantized_values, key=lambda x: x[1], reverse=True)
            sorted_quantized = [v for v,_ in sorted_random]
            if isprint: print(sorted_random)
            if isprint: print(sorted_quantized)
            
            while difference !=0:
                for i, value in sorted_quantized:
                    if difference == 0:
                        break
                    # Calculate adjustment
                    # adjustment = min(difference, value % (1 / factor))
                    # adjustment = difference
                    new_value = value - 1.0/factor
                    
                    # Update the quantized values
                    quantized_values = [(idx, new_value if idx == i else v) for idx, v in quantized_values]
                    difference -= 1.0/factor
                
        else:
            # # If the difference is negative (i.e., sum is less than 1), we need to increase values
            # sorted_quantized = sorted(quantized_values, key=lambda x: x[1])
            sorted_random = sorted(random_values, key=lambda x: (x[0][1],x[1]))
            # sorted_quantized = sorted(quantized_values, key=lambda x: x[1], reverse=True)
            sorted_quantized = [v for v,_ in sorted_random]
            
            while difference != 0:
                for i, value in sorted_quantized:
                    if difference == 0:
                        break
                    # Calculate adjustment
                    # adjustment = min(-difference, (1 / factor) - (value % (1 / factor)))
                    # adjustment = -difference
                    new_value = value + 1.0/factor
                    
                    # Update the quantized values
                    quantized_values = [(idx, new_value if idx == i else v) for idx, v in quantized_values]
                    difference += 1.0/factor
    
    # Return the list sorted by original indices
    # return [v for i, v in sorted(quantized_values)]
    if isprint: print(quantized_values)
    return quantized_values


with open(varfile,'r') as f:
    lines = f.readlines()

valuearr = list()
weightarr = list()
qweightarr = list()
for i in range(numsw):
    valuearr.append(list())
    weightarr.append(list())
    qweightarr.append(list())
    for j in range(numsw):
        valuearr[i].append(list())
        weightarr[i].append(list())
        qweightarr[i].append(list()) 

for line in lines:
    tokens = line.split(',')
    varname = tokens[0]
    vartokens = varname.split('_')
    fromsw = int(vartokens[0])
    tosw = int(vartokens[1])
    pid = int(vartokens[2])
    value = float(tokens[1])
    valuearr[fromsw][tosw].append([pid,value])
    weightarr[fromsw][tosw].append([pid,value])

for fromsw in range(numsw):
    for tosw in range(numsw):
        if len(valuearr[fromsw][tosw])>0:
            valuesum = 0
            for _,value in valuearr[fromsw][tosw]:
                valuesum += value
            valuetoweight = 1.0/valuesum
            for ipv,pv in enumerate(weightarr[fromsw][tosw]):
                weightarr[fromsw][tosw][ipv][1] = pv[1] * valuetoweight

myrseed = 0
for fromsw in range(numsw):
    for tosw in range(numsw):
        if len(weightarr[fromsw][tosw])>0:
            qweightarr[fromsw][tosw] = quantize_and_adjust_with_indices(weightarr[fromsw][tosw],factor,myrseed)
            myrseed += 1

qvarfile = f"/home/annzhou/DRing/src/emp/datacentre/qvarfiles/qvar_leafspine_{numserver}_{numsw}_{numport}_0_0_ecmp_{factor}"
with open(qvarfile,'w') as f:
    for fromsw in range(numsw):
        for tosw in range(numsw):
            if len(qweightarr[fromsw][tosw])>0:
                for pid,weight in qweightarr[fromsw][tosw]:
                    if weight>0:
                        f.write(f"{fromsw},{tosw},{pid},{weight}\n")

In [26]:
# check qvarfile
checkweightarr = list()
for i in range(numsw):
    checkweightarr.append(list())
    for j in range(numsw):
        checkweightarr[i].append(0)
with open(qvarfile,'r') as f:
    lines = f.readlines()
    for line in lines:
        tokens = line.split(',')
        fromsw = int(tokens[0])
        tosw = int(tokens[1])
        pid = int(tokens[2])
        weight = float(tokens[3])
        checkweightarr[fromsw][tosw] += weight

for fromsw in range(numsw):
    for tosw in range(numsw):
        if checkweightarr[fromsw][tosw]!=0 and checkweightarr[fromsw][tosw]!=1:
            print(f"from {fromsw} to {tosw}: weight={checkweightarr[fromsw][tosw]}")